# [Car Evaluation Dataset](https://archive.ics.uci.edu/dataset/19/car+evaluation)

Car Evaluation Database was derived from a simple hierarchical decision model originally developed for the demonstration of DEX, M. Bohanec, V. Rajkovic: Expert system for decision making. Sistemica 1(1), pp. 145-157, 1990.

### Estrutura do dataset

| Alvo  | Valores |
| ------------- | ------------- |
|class| {unacc, acc, good, vgood}|

| Features  | Valores |
| ------------- | ------------- |
|buying| {vhigh, high, med, low}|
|maint| {vhigh, high, med, low}|
|doors| {2, 3, 4, 5more}|
|persons| {2, 4, more}|
|lug_boot| {small, med, big}|
|safety| {low, med, high}|

### Instalação das dependências

In [15]:
# !pip install ucimlrepo
# !pip install tensorflow
# !pip install sklearn
# !pip install pandas

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten, Dense

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

from ucimlrepo import fetch_ucirepo 

### Importando o dataset

In [3]:
# fetch dataset 
car_evaluation = fetch_ucirepo(id=19) 
  
# data (as pandas dataframes) 
X = car_evaluation.data.features 
Y = car_evaluation.data.targets 

# Transform labels to int
labels = Y["class"].unique()
for i in range(len(labels)):
  Y.loc[Y['class']==labels[i], 'class'] = i


C:\Users\AMD RYZEN\AppData\Local\Temp\ipykernel_16880\464027119.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y.loc[Y['class']==labels[i], 'class'] = i


In [10]:
for feature in X:
    uniques = X[feature].unique()
    temp = ""
    for x in uniques:
        temp += x + ", "

    print(f"|{feature}| {{{temp}}}| ")

|buying| {vhigh, high, med, low, }| 
|maint| {vhigh, high, med, low, }| 
|doors| {2, 3, 4, 5more, }| 
|persons| {2, 4, more, }| 
|lug_boot| {small, med, big, }| 
|safety| {low, med, high, }| 


### One-hot Encoding das features

In [ ]:
X = pd.get_dummies(X, dtype=int)

### Dividindo os conjuntos de teste e treino

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size= 0.7, random_state = 28)

### One-Hot Enconding dos Targets

In [ ]:
y_train = to_categorical(y_train) 
y_true = list(y_test['class'])
y_test = to_categorical(y_test) 

### Perceptron

In [ ]:
model = tf.keras.Sequential([
    Flatten(input_shape=(len(X.columns), )),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(y_train[0]), activation='softmax')
])

In [ ]:
model.compile(
  loss='categorical_crossentropy', 
  optimizer='sgd', 
  metrics=['Accuracy', 'Precision', 'Recall', 'F1Score']
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 21)                0         
                                                                 
 dense (Dense)               (None, 128)               2816      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 4)                 260       
                                                                 
Total params: 11332 (44.27 KB)
Trainable params: 11332 (44.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Treinamento

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=32, verbose=1)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/200
17/17 [==============================] - 1s 2ms/step - loss: 1.2857 - Accuracy: 0.5270 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.3132
Epoch 2/200
17/17 [==============================] - 0s 1ms/step - loss: 1.1260 - Accuracy: 0.6892 - precision: 1.0000 - recall: 0.0019 - f1_score: 0.2165    
Epoch 3/200
17/17 [==============================] - 0s 1ms/step - loss: 1.0158 - Accuracy: 0.6931 - precision: 0.9605 - recall: 0.1409 - f1_score: 0.2047    
Epoch 4/200
17/17 [==============================] - 0s 1ms/step - loss: 0.9266 - Accuracy: 0.6931 - precision: 0.8521 - recall: 0.5116 - f1_score: 0.2047
Epoch 5/200
17/17 [==============================] - 0s 1ms/step - loss: 0.8662 - Accuracy: 0.6931 - precision: 0.7265 - recall: 0.6718 - f1_score: 0.2047
Epoch 6/200
17/17 [==============================] - 0s 1ms/step - loss: 0.8262 - Accuracy: 0.6931 - precision: 0.7002 - recall: 0.6853 - f1_score: 0.2047
Epoch 7/200
17/17 [==============================] - 0

[0.18009164929389954,
 0.9355372190475464,
 0.944209635257721,
 0.9231404662132263,
 array([0.975754  , 0.88454705, 0.7654321 , 0.6352942 ], dtype=float32)]

In [ ]:
predictions = model.predict(x_test)
predictions = [list(p).index(max(p)) for p in predictions]


38/38 [==============================] - 0s 622us/step


In [ ]:
print(f'Acurácia obtida: {accuracy_score(y_true, predictions) * 100:.2f}%')
print(f'Precisão obtida: {precision_score(y_true, predictions, average="weighted") * 100:.2f}%')
print(f'Recall obtido: {recall_score(y_true, predictions, average="weighted") * 100:.2f}%')
print(f'F1 Score obtida: {f1_score(y_true, predictions, average="weighted") * 100:.2f}%')


Acurácia obtida: 93.55%
Precisão obtida: 93.79%
Recall obtido: 93.55%
F1 Score obtida: 93.56%
